In [ ]:
!pip install -q -U google-genai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-flash-latest')

try:
    response = model.generate_content("Hello! Explain what you are in one sentence.")
    print("AI Response:")
    print(response.text)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
campus_persona = """
You are the official AI Campus Assistant for VIT Vellore.
Your goal is to help students with simple queries about campus life.
Guidelines:
- Be friendly, encouraging, and professional.
- Keep answers short (max 2-3 sentences).
- If you don't know something, say "I don't have that info yet."
"""

model = genai.GenerativeModel('gemini-flash-latest', system_instruction=campus_persona)

chat = model.start_chat(history=[])

print("🤖 VIT Campus Assistant is ready! (Type 'quit' to stop)")
print("-" * 50)

while True:
    user_input = input("Student: ")

    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("AI: Goodbye! Study hard!")
        break

    response = chat.send_message(user_input)

    print(f"AI: {response.text}")
    print("-" * 50)

In [ ]:
!pip install -q chromadb langchain-google-genai langchain-community langchain-text-splitters

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

handbook_text = """
VIT VELLORE CAMPUS GUIDE 2026
1. EXAM SCHEDULE:
   - CAT-1 Exams: January 27th to February 2nd, 2026.
   - CAT-2 Exams: March 15th to March 23rd, 2026.
   - FAT (Final) Exams: April 21st to May 3rd, 2026.
2. LIBRARY RULES:
   - The Periyar Library is open 24/7 during exam weeks.
   - Normal hours: 8:00 AM to 12:00 Midnight.
   - Quiet zones are strictly enforced on the 3rd floor.
3. DRESS CODE:
   - Students must wear ID cards at all times within academic blocks.
   - Formal or smart-casual wear is required for labs.
4. BUS TIMINGS:
   - Shuttle buses run every 15 minutes between the Main Gate and SJT.
   - First bus: 7:30 AM. Last bus: 8:30 PM.
"""

docs = [Document(page_content=handbook_text)]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

print("✅ Success! Stored chunks in ChromaDB vector store.")

In [ ]:
!pip install -q langchain-huggingface sentence-transformers chromadb langchain-community

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

print("📥 Downloading local embedding model... (this happens only once)")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

handbook_text = """
VIT VELLORE CAMPUS GUIDE 2026
1. EXAM SCHEDULE:
   - CAT-1 Exams: January 27th to February 2nd, 2026.
   - CAT-2 Exams: March 15th to March 23rd, 2026.
   - FAT (Final) Exams: April 21st to May 3rd, 2026.

2. LIBRARY RULES:
   - The Periyar Library is open 24/7 during exam weeks.
   - Normal hours: 8:00 AM to 12:00 Midnight.
   - Quiet zones are strictly enforced on the 3rd floor.

3. DRESS CODE:
   - Students must wear ID cards at all times within academic blocks.
   - Formal or smart-casual wear is required for labs.

4. BUS TIMINGS:
   - Shuttle buses run every 15 minutes between the Main Gate and SJT.
   - First bus: 7:30 AM. Last bus: 8:30 PM.
"""

with open("vit_handbook.txt", "w") as f:
    f.write(handbook_text)

docs = [Document(page_content=handbook_text)]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

print("✅ Success! Stored chunks in ChromaDB vector store (Locally!).")

In [ ]:
model = genai.GenerativeModel('gemini-flash-latest')

def ask_campus_bot(question):
    print(f"🔎 Searching handbook for: '{question}'...")

    relevant_docs = vectorstore.similarity_search(question, k=2)

    context_text = ""
    for i, doc in enumerate(relevant_docs):
        context_text += f"\n[Chunk {i+1}]: {doc.page_content}\n"
    print(f"📄 Found Context:{context_text}")

    prompt = f"""
    You are the VIT Campus Assistant.
    Answer the student's question strictly based on the context provided below.
    If the answer is not in the context, say "I don't find that in the handbook."

    CONTEXT:
    {context_text}

    QUESTION:
    {question}
    """

    response = model.generate_content(prompt)
    print(f"🤖 AI Answer: {response.text}")
    print("-" * 50)

ask_campus_bot("When are the CAT-1 exams?")
ask_campus_bot("What is the dress code for labs?")
ask_campus_bot("Can I bring my pet tiger?")

In [ ]:
def search_handbook(query):
    """
    Searches the VIT Student Handbook for rules, dates, and guidelines.
    Use this tool ONLY when the user asks about campus facts.
    """
    print(f"\n[System] 🛠️  Tool Triggered: Searching for '{query}'...")

    relevant_docs = vectorstore.similarity_search(query, k=2)

    context_text = "\n".join([f"Rule: {doc.page_content}" for doc in relevant_docs])
    return context_text

tools_list = [search_handbook]

print("✅ Tool 'search_handbook' created and ready for the Agent.")

In [ ]:
def log_complaint(student_issue):
    """Saves a student's complaint to a text file."""
    with open("complaints.txt", "a") as f:
        f.write(f"ISSUE: {student_issue}\n")
    return "Complaint saved successfully."

In [ ]:
agent_model = genai.GenerativeModel(
    'gemini-flash-latest',
    tools=[search_handbook, log_complaint],
    system_instruction="""
    You are the VIT Campus Agent.
    - You have a tool called 'search_handbook'.
    - ALWAYS use this tool if the user asks about rules, exams, or campus facts.
    - If a student seems unhappy or reports a problem, use the log_complaint tool.
    - If the user just says 'Hi' or 'Thanks', reply naturally without the tool.
    """
)

chat_agent = agent_model.start_chat(enable_automatic_function_calling=True)

print("🕵️‍♂️ Agent is online! (Type 'quit' to exit)")
print("-" * 50)

while True:
    user_input = input("Student: ")
    if user_input.lower() in ['quit', 'exit']:
        print("Agent: Goodbye!")
        break

    try:
        response = chat_agent.send_message(user_input)
        print(f"Agent: {response.text}")
        print("-" * 50)
    except Exception as e:
        print(f"Error: {e}")